Author: Hasim Engin <br>
email:hengin@ciesin.columbia.edu<br>
Project: GRID3<br>
Organization:CIESIN, Columbia University<br>

# Create Catchments for a PoI 

This notebook calculates dictance from s settlement to the cloesest PoI by linear distance based on a friction surface.
## Input layers:

* **Friction layers**: Friction layers (walk and walk+motorized) in minutes.They can be created by running "create_friction_surface" script.
* **PoI layer**: A point of interest layer. Such as health facilities, schools, water sources .... POI layer must have unique id and capacity for each record
* **Settlement extent layer**: This is a vector layer produced by GRID3 for sub-saharas countries. They can be downloaded here:https://grid3.org/resources/data



## Output layers: 

Output layers will be saved into a directory that is created specified workspace directory.The scripts goes trought individual admin units that is specified and calculates distance between each individual settlement to the closest PoI.
Individual admin units result will be saved in the output.gdb. Then all the admin level settlement extent will merged and saved in the final.gdb. Output.gdb can be deleted if output layer in the final.gdb is good.
 <br>
* **output.gbd** : admin level settlement extent will be saved here. 
 <br>
* **Final.gdb** : <br>
   ..._settlement_extent: Settlement extent with distance to closest PoI in meter.<br>
   ..._settlement_part: The shortest path that connects a settlement to a health facility. Distance is calculated based on these paths. 
   
    
    


## import Libraries

In [6]:

# import libraries
import os
import arcpy
import re
import unidecode
import pandas as pd
import arcpy.cartography as CA
from arcgis.features import SpatialDataFrame
arcpy.env.overwriteOutput = True



In [ ]:
# required libraries can be installed with following code
#uncommaent the line below and write a -library name- after !):
#! -library name- install

## Functions

In [7]:


###==================================== functions =============================================###

def calculate_shorthest_distrance(admin_name):
    admin_name=str(admin_name)
        ##Creates cathment areas along with some popualtion summary
        ##Cathments areas are nested in the admin units
       
    if not arcpy.Exists("settlement_extent_"+admin_name):
       
        admin_boundary="boundary_"+admin_name
        admin_poi="poi_"+admin_name
        admin_sett="sett_"+admin_name
        admin_sett_point="sett_point_"+admin_name
        admin_friction="friction_"+admin_name
        
        
        ##==========================================================================================###
        cost_distance="accessibility_"+admin_name
        catchment_areas_update="catchments_"+  admin_name+"_"+poi_subtype
        sett_path="sett_path_"+ admin_name+"_"+poi_subtype
        sett_path_smoothline="sett_path_smoothlines_"+admin_name+"_"+poi_subtype
        sett_with_cost="Sett_extent_cost_in_min_"+ admin_name +"_"+poi_subtype

        ##==========================================================================================### 

        ## extract layers by the admin unit
        arcpy.Select_analysis(path_to_admin_boundary,admin_boundary,"OBJECTID = {}".format(admin_name))
        arcpy.analysis.Buffer(admin_boundary,"buffer", "100 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")
        arcpy.Select_analysis(path_to_poi,admin_poi,"NEAR_FID = {}".format(admin_name))

        arcpy.gp.ExtractByMask_sa(friction_layer,"buffer",admin_friction)
 
        
        arcpy.Clip_analysis(path_to_sett_extent, admin_boundary,admin_sett)
        arcpy.FeatureToPoint_management(admin_sett,admin_sett_point, "INSIDE")

        check_bdry=int(arcpy.GetCount_management(admin_boundary).getOutput(0))
        point_check=int(arcpy.GetCount_management(admin_poi).getOutput(0))
        print (check_bdry,point_check)
        if  check_bdry>=1 and point_check>=1 :


            print (f"Creating accessiblility layers for =={country_iso}== for =={poi_type}== with =={travel_type}== travel mode" )
            print(" ")
            print ( "   >>> Creating cost distance in minutes to PoI")
            arcpy.sa.CostDistance(admin_poi,admin_friction,"" , "outBkLinkRaster").save(  cost_distance)


            #---------------------------------------------------------------------------------#

            print ("   >>> Creating cost path from settlements to PoI")
            arcpy.sa.CostPathAsPolyline(admin_sett_point, cost_distance, "outBkLinkRaster", sett_path, "EACH_ZONE", "OBJECTID", "INPUT_RANGE")
            arcpy.AddField_management(sett_path, "dist_in_meter", "DOUBLE")
            arcpy.CalculateGeometryAttributes_management(sett_path, "dist_in_meter LENGTH_GEODESIC", "METERS")
            sdf_path=pd.DataFrame.spatial.from_featureclass(sett_path)[["PathCost","DestID","dist_in_meter"]]
            sdf_path.rename(columns={"PothCost":"dist_in_min","DestID":"matchid"}, inplace=True)
            print ("   >>> Smoothing paths")
            CA.SimplifyLine(sett_path,sett_path_smoothline, "POINT_REMOVE", 100,"","NO_KEEP")

            print ( "   >>> Calculating everage travel time in minutes to closest PoI...")
            arcpy.RasterToPoint_conversion(cost_distance, "cost_distance_point", "VALUE")
            spatial_join(admin_sett, "cost_distance_point", "average_cost_min", sett_with_cost)
            sdf_sett=pd.DataFrame.spatial.from_featureclass(sett_with_cost)[["OBJECTID_1","OBJECTID","MGRS_Code","average_cost_min", "SHAPE"]]
            sdf_sett.rename(columns={"OBJECTID":"matchid","average_cost_min":"avrg_dist_min"}, inplace=True)
            sdf_merge=sdf_sett.merge(sdf_path, how="left",on="matchid")
            sdf_merge.spatial.to_featureclass(output_gdb+"\\"+"settlement_extent_"+admin_name)

            print ("   >>> Deleting temporary layers...")
            delete_fc=["sett_point_"+admin_name,"boundary_"+admin_name,"buffer","friction_"+admin_name,"outBkLinkRaster","poi_"+admin_name,"sett_"+admin_name,
                       "accessibility_"+admin_name,"Sett_extent_cost_in_min_"+admin_name+"_all"]
            for fc in delete_fc:
                 arcpy.Delete_management(fc)      
            print ("##====================================================================================================##")    


def merge_outputs(select_layer_to_be_merged, output_name):
    merge_fc=[fc for fc in arcpy.ListFeatureClasses() if fc.startswith(select_layer_to_be_merged)]
    if len(merge_fc)>1:
        arcpy.Merge_management(merge_fc, final_gdb+"\\"+output_name)




## User Inputs

In [9]:

###=================== Initialize Variables ===================###

country_iso=""          # three latters iso code
poi_type=  ""           # type of point of interest such as "health_facility"      !! no space between names !!
travel_type= ""         # options: walk or mix( walk+motorized)                      
admin_col=""            # admin column in admin boundary layer.
###================================ input layers ==============================================###

# path to PoI layer such as health facilities or schools
path_to_poi=r""

# friction surface for walking. It is created by "create_friction_surface" script
path_to_friction_walk=r""

# friction surface for motorized.It is created by "create_friction_surface" script
path_to_friction_mix=r""

#path to settlement extent layer
path_to_sett_extent==r"" 

#path to admin  boundary
path_to_admin_boundary=r""

# path to the directory that you want to save outputs
output_path=r""

## Create Output Workspace

In [11]:
 #create output directory
if not  os.path.exists (os.path.join(output_path,"output")):
    os.mkdir(os.path.join(output_path,"output"))
output_loc=os.path.join(output_path,"output")

 #create output gdb
if  not arcpy.Exists(os.path.join(output_loc,"output.gdb")):
    arcpy.CreateFileGDB_management(output_loc,"output.gdb")
output_gdb=os.path.join(output_loc,"output.gdb")
 #create final gdb
if  not arcpy.Exists(os.path.join(output_loc,"final.gdb")):
    arcpy.CreateFileGDB_management(output_loc,"final.gdb")
final_gdb=os.path.join(output_loc,"final.gdb")

# workspace
arcpy.env.workspace=output_gdb

## Process

In [13]:


###==================== select friction layer based on travel type============================###

if travel_type=="walk":
    friction_layer= path_to_friction_walk
if travel_type=="mix":
    friction_layer= path_to_friction_mix
    
###=================== transfer admin name from admin boundary to PoI =======================###   
# get admin names ( no special characters)   and respective abjectids
objectid_list=[]
admin_list=[]
arcpy.AddField_management(path_to_admin_boundary, "admin_cln", "TEXT")
with arcpy.da.UpdateCursor(path_to_admin_boundary,[admin_col,"admin_cln", 'OBJECTID']) as cursor:
    for row in cursor:
        if row[0] is not None:
            fix_admin= unidecode.unidecode(row[0])
            fix_admin=re.sub(r'[-|$|.|!|)|(|!|-|_|]',r'',fix_admin) # spatial characters need to be removed in admin names
            fix_admin=fix_admin.strip().replace("  ","_").replace(" ","_")
            row[1]=fix_admin
            admin_list_.append(row[1])
            objectid_list.append(row[2])
            cursor.updateRow(row)

            
 ##Transfer admin names from admin boundary to PoI layer.
 ##Admin names in boudary laeyr and PoI layer has to be identical
arcpy.Near_analysis(path_to_poi,path_to_admin_boundary,"10 Centimeters")
arcpy.AddField_management(path_to_poi, "admin_cln", "TEXT")
for i, name_ in zip(objectid_list,admin_list_): 
    with arcpy.da.UpdateCursor(path_to_poi,["NEAR_FID","admin_cln"]) as cursor:
        for row in cursor:
            if row[0]==i:
                row[1]=name_
                admin_list2_.append(i)
            cursor.updateRow(row)

admin_list=list(set(admin_list)) 
###======================== prosessing ==============================###

not_run=[]

count=len(admin_list)
## Create output by each admin units
for admin in admin_list:
    print(count)
    try:
        print ("##########  "+str(admin)+"  ###########")
        create_accessibility_layers2(admin)
    except:
        print (str(admin)+"  did NOT RUN")
        not_run.append(admin)
    count=count-1

("   >>> Merging district based output layers...")
merge_outputs("sett_path_smoothlines_",country_iso+"_"+poi_type+"_"+travel_type+"_settlement_path" )
merge_outputs("settlement_extent_",country_iso+"_"+poi_type+"_"+travel_type+"_settlement_extent" )


# get if any admin unit did not run
if len (not_run) >=1:
    print ("These admin units did not run properly")
    for i in not_run:
        print (i)
print ("SCRIPT IS DONE")
    



